***GENERATED CODE FOR dp1 PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***OPERATION FUNCTIONS***

In [ ]:
import pyspark
from dask.dataframe import from_pandas
import json


def calculateFormula(df, functionsData, applyOn):
    dfcp = df
    for functionData in functionsData:
        if functionData['useNext']:
            if 'dataPrepColumn_Inprocess' in list(dfcp.columns):
                dfcp['dataPrepColumn_Inprocess'] = calculateEquation(
                    dfcp, functionData['operator'], functionData['dataPrepColumn_Inprocess'], functionData['operand2'])
            else:
                dfcp['dataPrepColumn_Inprocess'] = calculateEquation(
                    dfcp, functionData['operator'], functionData['operand1'], functionData['operand2'])
        else:
            if 'dataPrepColumn_Inprocess' in list(dfcp.columns):
                if functionData['outputColumn'] == "newColumn":
                    newColumnName = "New_" + \
                        functionData['operand1']+"_" + \
                        functionData['operator']+"_"+functionData['operand2']
                    dfcp[newColumnName] = calculateEquation(dfcp, functionData['operator'],
                                                            functionData['dataPrepColumn_Inprocess'], functionData['operand2'])
                else:
                    dfcp[functionData['outputColumn']] = calculateEquation(dfcp, functionData[
                        'operator'], functionData['dataPrepColumn_Inprocess'], functionData['operand2'])
                dfcp.drop('dataPrepColumn_Inprocess', axis=1)
            else:
                if functionData['outputColumn'] == "newColumn":
                    newColumnName = "New_" + \
                        functionData['operand1']+"_" + \
                        functionData['operator']+"_"+functionData['operand2']
                    dfcp[newColumnName] = calculateEquation(dfcp, functionData[
                        'operator'], functionData['operand1'], functionData['operand2'])
                else:
                    dfcp[functionData['outputColumn']] = calculateEquation(dfcp, functionData[
                        'operator'], functionData['operand1'], functionData['operand2'])
    return dfcp


def calculateEquation(df, operator, columnA, columnB):
    if (operator == "add"):
        return df[columnA].add(df[columnB])
    elif (operator == "sub"):
        return df[columnA].sub(df[columnB])
    elif (operator == "mul"):
        return df[columnA].mul(df[columnB])
    elif (operator == "div"):
        return df[columnA].div(df[columnB])
    elif (operator == "mod"):
        return df[columnA].mod(df[columnB])


def appendValueToColumn(df, functionsData, applyOn):
    for obj in functionsData:
        df[obj['columnName']] = df[obj['columnName']].astype(
            str) + obj['appendValue']
    return df


def runDataCleansing(sparkDf, spark, config):
    configObj = json.loads(config)
    sparkDf.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = from_pandas((sparkDf.toPandas()), npartitions=5)
    functionList = configObj['functionsApplied']
    Data_Cleansing_Methods = {"replaceBy": replaceValues,
                              "formula": calculateFormula,
                              "aggregate": aggregation,
                              "converttostringtype": changeToString,
                              "editname": renameColumns}
    for function in functionList:
        function['functionName']
        df = Data_Cleansing_Methods[function['functionName']](df, function['functionsData'],
                                                              function['applyOn'])
    sparkDf = spark.createDataFrame(df.compute())

    display(sparkDf.limit(2).toPandas())
    return sparkDf


***CONNECTOR FUNCTIONS TO WRITE DATA.***

In [ ]:
import datetime
import requests
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class NumtraConnector:

    def put(inStages, inStagesData, stageId, spark, config):
        path = eval(config)['server_url']
        baseType = eval(config)['baseType']
        results_url = eval(config)['results_url']
        server = eval(config)['server']
        originalfile = eval(config)['orignalKey']
        eval(config)['pathOnly']
        filename = eval(config)['filename']
        eval(config)['ser']
        eval(config)['user']
        eval(config)['password']
        eval(config)['authSource']
        eval(config)['user_id']
        eval(config)['parent_id']
        eval(config)['project_id']
        time = str(int(datetime.datetime.now().timestamp()))

        inStagesData[inStages[0]]

        print(path)
        print(baseType)
        print(results_url)
        print(server)
        print(originalfile)
        print(filename)

        args = {
            'url': path,
            'baseType': baseType,
            'originalfile': originalfile,
            'filename': time + filename
        }

        response = requests.post(results_url, args)
        return response


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://0.0.0.0:0000').getOrCreate()
#%run dp1Hooks.ipynb
try:
	#sourcePreExecutionHook()

	customerchurn = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/CustomerChurn.csv', 'filename': 'CustomerChurn.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'results_url': 'http://ml.numtra.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)


***PERFORMING OPERATIONS***

In [ ]:
#%run dp1Hooks.ipynb
try:
	#operationPreExecutionHook()

datapreparation = runDataCleansing(customerchurn,spark,json.dumps( {"url": "/FileStore/platform/uploadedSourceFiles/CustomerChurn.csv", "source_attributes": {}, "DataPrepFile": "/FileStore/platform/uploadedSourceFiles/CustomerChurn.csv", "data_source": "localfiles", "startListenerOnly": 1, "dateColumnNames": [], "FilePath": "/FileStore/platform/extra/64a66af11ba55170f7a206c11688628654/0part.csv", "requestRatio": 14.0, "totalRows": 7043, "BasicStats": {"missingValues": 0.0, "numberOfColumns": 23, "numberOfRows": 7043, "duplicateRowCount": 0, "stats": [{"column": "customerID", "alias": "customerID", "generated": 0, "type": "String", "max": "9995-HOTOH", "min": "0002-ORFBO", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "gender", "alias": "gender", "generated": 0, "type": "String", "max": "Male", "min": "Female", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "SeniorCitizen", "alias": "SeniorCitizen", "generated": 0, "type": "numeric", "max": 1, "min": 0, "mean": 0.1621468124378816, "missing": 0.0, "stddev": 0.37, "outliers": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], "validation": []}, {"column": "Partner", "alias": "Partner", "generated": 0, "type": "String", "max": "Yes", "min": "No", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "Dependents", "alias": "Dependents", "generated": 0, "type": "String", "max": "Yes", "min": "No", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "tenure", "alias": "tenure", "generated": 0, "type": "numeric", "max": 72, "min": 0, "mean": 32.37114865824223, "missing": 0.0, "stddev": 24.56, "outliers": [], "validation": []}, {"column": "PhoneService", "alias": "PhoneService", "generated": 0, "type": "String", "max": "Yes", "min": "No", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "MultipleLines", "alias": "MultipleLines", "generated": 0, "type": "String", "max": "Yes", "min": "No", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "InternetService", "alias": "InternetService", "generated": 0, "type": "String", "max": "No", "min": "DSL", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "OnlineSecurity", "alias": "OnlineSecurity", "generated": 0, "type": "String", "max": "Yes", "min": "No", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "OnlineBackup", "alias": "OnlineBackup", "generated": 0, "type": "String", "max": "Yes", "min": "No", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "DeviceProtection", "alias": "DeviceProtection", "generated": 0, "type": "String", "max": "Yes", "min": "No", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "TechSupport", "alias": "TechSupport", "generated": 0, "type": "String", "max": "Yes", "min": "No", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "StreamingTV", "alias": "StreamingTV", "generated": 0, "type": "String", "max": "Yes", "min": "No", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "StreamingMovies", "alias": "StreamingMovies", "generated": 0, "type": "String", "max": "Yes", "min": "No", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "Contract", "alias": "Contract", "generated": 0, "type": "String", "max": "Two year", "min": "Month-to-month", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "PaperlessBilling", "alias": "PaperlessBilling", "generated": 0, "type": "String", "max": "Yes", "min": "No", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "PaymentMethod", "alias": "PaymentMethod", "generated": 0, "type": "String", "max": "Mailed check", "min": "Bank transfer (automatic)", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "MonthlyCharges", "alias": "MonthlyCharges", "generated": 0, "type": "real", "max": 118.75, "min": 18.25, "mean": 64.76169246059918, "missing": 0.0, "stddev": 30.09, "outliers": [], "validation": []}, {"column": "TotalCharges", "alias": "TotalCharges", "generated": 0, "type": "real", "max": 8684.8, "min": 18.8, "mean": 2283.3004408418656, "missing": 0.2, "stddev": 2266.77, "outliers": [8129.3, 8041.65, 8126.65, 8468.2, 8404.9, 8399.15, 8248.5, 8220.4, 8071.05, 8306.05, 8196.4, 8065.65, 8476.5, 8310.55, 8046.85, 8333.95, 8093.15, 8317.95, 8127.6, 8375.05, 8100.55, 8078.1, 8086.4, 8477.7, 8477.6, 8564.75, 8124.2, 8529.5, 8058.85, 8164.1, 8425.3, 8496.7, 8405.0, 8277.05, 8165.1, 8182.85, 8182.75, 8308.9, 8240.85, 8061.5, 8297.5, 8109.8, 8425.15, 8058.55, 8436.25, 8349.45, 8456.75, 8443.7, 8312.4, 8424.9, 8100.25, 8672.45, 8684.8, 8332.15, 8312.75, 8289.2, 8166.8, 8075.35, 8594.4, 8244.3, 8543.25, 8331.95, 8250.0, 8192.6, 8547.15, 8175.9, 8309.55, 8337.45, 8152.3, 8349.7, 8670.1], "validation": []}, {"column": "Churn", "alias": "Churn", "generated": 0, "type": "String", "max": "Yes", "min": "No", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}, {"column": "New Charges", "alias": "New Charges", "generated": 1, "type": "real", "max": 1.0, "min": 0.012603701340140395, "mean": 0.1575918928686763, "missing": 0.2, "stddev": 0.28, "outliers": [1.0, 0.49791955617198336, 0.4662050774810419, 1.0, 1.0, 1.0, 0.5257363060831268, 1.0, 1.0, 0.5077319587628866, 0.5594866458550122, 0.308480131093814, 1.0, 0.36358511837655016, 1.0, 1.0, 1.0, 1.0, 0.24251497005988024, 0.3617149758454106, 0.3241123938139839, 0.33147853736089034, 1.0, 1.0, 0.5458461538461539, 1.0, 0.319478402607987, 1.0, 0.5155325443786983, 0.25970695970695973, 0.4299835255354201, 1.0, 1.0, 1.0, 0.3700547778454047, 1.0, 1.0, 0.5443786982248521, 1.0, 0.26141233463760016, 1.0, 0.27906328408140507, 1.0, 1.0, 0.476774758983348, 0.27560351133869787, 1.0, 0.4784233775059463, 1.0, 0.553693358162632, 0.3112708018154312, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.24097450138853824, 0.2534697883078649, 1.0, 1.0, 0.5059331175836029, 0.5173299101412066, 1.0, 0.5374554102259216, 0.24119402985074626, 1.0, 0.471900826446281, 1.0, 1.0, 1.0, 1.0, 0.49608062709966405, 1.0, 1.0, 0.3480958493795464, 1.0, 0.4472803347280335, 1.0, 1.0, 0.5263157894736842, 1.0, 0.26211453744493396, 1.0, 1.0, 1.0, 0.5168637757336837, 0.533715925394548, 1.0, 0.5623880597014925, 1.0, 0.45829750644883926, 1.0, 1.0, 1.0, 1.0, 0.26501965188096577, 0.5064432989690721, 1.0, 0.25765453495089546, 1.0, 0.3241491085899514, 1.0, 0.23619271445358406, 1.0, 0.27634907251264756, 0.4182389937106918, 1.0, 1.0, 0.5541795665634675, 0.31286549707602335, 1.0, 0.5529247910863511, 0.25247267048412286, 1.0, 0.3139437211255775, 0.47984843265587324, 0.2389426851609526, 1.0, 0.5574297188755021, 0.3442283163265306, 1.0, 0.32915921288014316, 1.0, 1.0, 0.5406147091108672, 1.0, 0.2774288880679719, 1.0, 0.2982456140350877, 0.6116071428571429, 0.23626769009354762, 1.0, 0.581294964028777, 0.3204946996466431, 0.5867979576951131, 0.36988664366683094, 0.4742782152230971, 0.34702797202797203, 1.0, 0.3366052190567393, 1.0, 0.43567567567567567, 0.4440834010289737, 0.5067039106145251, 1.0, 0.4469195962408632, 1.0, 1.0, 0.4823726916620034, 0.26929644808743164, 1.0, 1.0, 0.40080645161290324, 1.0, 0.30936890045543264, 0.2525458248472505, 0.30539710399297937, 1.0, 1.0, 0.7078039927404719, 0.2401532893336172, 1.0, 0.3872226286271641, 0.23493474034990278, 0.2702483363090407, 0.3447636700648749, 0.26417525773195877, 0.5278919553728714, 1.0, 1.0, 0.3437757625721352, 0.5491990846681922, 1.0, 1.0, 1.0, 1.0, 0.5309667673716012, 0.24412074412074414, 0.5757828810020877, 0.31178116257382654, 1.0, 0.24653739612188366, 0.3225570776255708, 0.329940906106369, 1.0, 0.48261924009700885, 0.4825319958491871, 0.5384193194291987, 1.0, 0.34740259740259744, 0.47297692082968157, 1.0, 0.33392804044008323, 0.243801652892562, 0.3981385729058945, 1.0, 0.3533724340175953, 0.48324022346368717, 1.0, 0.35884847132336534, 1.0, 1.0, 1.0, 1.0, 0.3337877312560857, 1.0, 0.3612255406797117, 1.0, 0.3211357728454309, 0.5294733170561563, 1.0, 0.33690893169877406, 0.3481613285883749, 1.0, 1.0, 1.0, 1.0, 0.5232853513971211, 0.5423180592991913, 0.41197635569262403, 0.2642410273296016, 1.0, 1.0, 0.2848870874348582, 1.0, 0.26653569089718404, 1.0, 1.0, 1.0, 0.4665629860031104, 1.0, 0.3228090882266156, 0.23769125890016665, 0.29501996217692794, 1.0, 1.0, 0.5472136222910218, 1.0, 1.0, 0.25923028785982477, 0.3421955403087479, 1.0, 0.5493122622183202, 1.0, 0.472107438016529, 0.36104820446457453, 1.0, 0.24304406677695894, 1.0, 0.2505338078291815, 1.0, 0.5502717391304348, 0.5357664233576642, 1.0, 1.0, 0.3295362082994304, 1.0, 1.0, 0.5883977900552486, 0.3547511312217195, 0.32543066689408157, 0.473559589581689, 1.0, 1.0, 0.44922737306843274, 1.0, 0.26504872921842154, 1.0, 0.35405192761605037, 0.45163240628778717, 0.585031185031185, 1.0, 0.6491620111731844, 1.0, 0.31730551145384445, 1.0, 1.0, 1.0, 0.3293680297397769, 1.0, 0.25058621228700956, 1.0, 0.3221299093655589, 1.0, 0.37601832993890016, 0.2482482482482483, 0.33882611424984305, 0.5119120342089187, 0.34736456808199123, 1.0, 1.0, 1.0, 1.0, 1.0, 0.4742857142857143, 0.3199924485557863, 0.560840024706609, 0.5499477897667943, 1.0, 0.5409495548961425, 1.0, 0.2625806451612903, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5105454545454545, 1.0, 1.0, 1.0, 1.0, 0.33872598584428715, 1.0, 1.0, 0.30923994038748137, 1.0, 0.519794499848897, 1.0, 1.0, 0.2457197974439354, 0.25962165688193084, 0.3448275862068965, 0.37083811710677383, 1.0, 1.0, 0.2664067576348278, 1.0, 0.32089249492900607, 1.0, 1.0, 0.2890005288207298, 1.0, 0.50642834744434, 1.0, 1.0, 1.0, 1.0, 1.0, 0.45932028836251293, 0.44284925455549423, 0.2550885927661444, 1.0, 0.4528231797919763, 1.0, 1.0, 1.0, 1.0, 0.3754547659471259, 0.4636320976067574, 1.0, 1.0, 1.0, 1.0, 0.24410591222343125, 1.0, 1.0, 0.51931330472103, 0.3109504132231405, 0.3062926459438969, 1.0, 1.0, 0.2485059760956175, 1.0, 1.0, 1.0, 1.0, 0.34179357021996615, 1.0, 0.43753394894079306, 0.47153965785381025, 0.546229802513465, 1.0, 0.2792924375202856, 0.4594965675057209, 0.4940027894002789, 1.0, 1.0, 1.0, 0.23963290278721958, 1.0, 1.0, 1.0, 0.3681912204091013, 0.5491270112975009, 1.0, 0.3930753564154786, 1.0, 1.0, 1.0, 0.3548079974611234, 0.266283227110213, 1.0, 1.0, 0.5170362358031367, 0.3495121341005754, 0.28336755646817247, 0.24065138721351023, 0.3542066845946984, 0.23450292397660819, 0.31658786446020487, 1.0, 1.0, 1.0, 1.0, 1.0, 0.38339920948616596, 0.32926152137701276, 1.0, 1.0, 0.4745098039215686, 0.4998232591021563, 0.2469570787956438, 1.0, 1.0, 0.5147368421052632, 0.3239901071723001, 0.50138418948016, 0.25284428595686875, 1.0, 1.0, 1.0, 0.2415422885572139, 0.351439656932816, 0.5013927576601671, 0.40010570824524316, 1.0, 1.0, 0.5333916593759114, 1.0, 0.5949554896142433, 0.5161290322580645, 0.3207480592801694, 1.0, 1.0, 0.32808455565142364, 1.0, 1.0, 1.0, 1.0, 0.42822966507177035, 1.0, 0.5024875621890548, 1.0, 1.0, 0.48899755501222497, 1.0, 1.0, 0.24504307397784766, 0.5451023796347537, 1.0, 0.26245784938179095, 0.2476775049767751, 0.24394966118102615, 0.3846899224806202, 1.0, 1.0, 0.5032938076416338, 0.3361144219308701, 1.0, 0.5280172413793104, 0.5047114252061249, 1.0, 1.0, 1.0, 0.2555938037865748, 0.2808664259927798, 1.0, 0.38554778554778557, 1.0, 1.0, 0.24753974261922784, 0.23828647925033467, 0.3059554477951205, 0.5272875373336953, 0.33714939896966223, 0.2501699524133243, 1.0, 1.0, 1.0, 1.0, 0.31453634085213034, 0.23856613102595797, 0.2864950381130447, 0.5495137046861185, 1.0, 1.0, 1.0, 0.28183023872679047, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.3626570915619389, 0.24768661320172733, 1.0, 0.3261441346659653, 0.33674799776494696, 0.30802437785678005, 1.0, 1.0, 1.0, 1.0, 0.5162292817679558, 1.0, 1.0, 0.2819270966097374, 0.5229357798165137, 1.0, 1.0, 1.0, 0.3865225683407501, 1.0, 0.5374848851269649, 0.33559898045879355, 1.0, 0.4563883702320619, 0.2627751196172249, 0.6319895968790636, 0.2601938502673797, 0.32590873936581594, 1.0, 0.2593295382669197, 1.0, 0.4755244755244755, 1.0, 1.0, 0.24081115335868186, 1.0, 1.0, 0.3948412698412699, 0.5276005547850208, 1.0, 0.2825018615040953, 0.31446540880503143, 0.3890074706510139, 1.0, 0.36730769230769234, 0.28441011235955055, 1.0, 1.0, 1.0, 0.4817432273262661, 0.307176045431079, 0.40432237730751913, 0.3156779661016949, 0.2528541226215645, 1.0, 0.34652843229657554, 1.0, 0.2525986224170319, 1.0, 0.24948778565799845, 0.2578268876611418, 1.0, 0.24069627851140457, 1.0, 0.25511372934053345, 0.33153750518027353, 1.0, 1.0, 1.0, 0.48162014976174267, 1.0, 0.25272941870758336, 1.0, 1.0, 0.45176027949475944, 1.0, 1.0, 1.0, 1.0, 0.47632575757575757, 0.5263915547024952, 1.0, 0.43468468468468474, 1.0, 1.0, 0.2586793737236215, 0.2504222972972973, 0.24683162341581172, 0.33509583608724386, 0.28298887122416533, 0.4559386973180077, 1.0, 1.0, 0.45198098256735336, 0.2676976693530821, 1.0, 1.0, 1.0, 1.0, 0.6363636363636365, 0.23630855278489862, 0.5589469668065623, 0.2964467005076142, 0.5301478953356086, 1.0, 0.45058517555266575, 1.0, 0.25223499361430396, 1.0, 0.35018196522442374, 1.0, 1.0, 0.529621698786581, 1.0, 1.0, 1.0, 1.0, 0.346460081773187, 0.47481420313790257, 1.0, 0.6054333764553687, 0.3350484261501211, 1.0, 0.5660493827160494, 0.667578659370725, 0.24502923976608187, 1.0, 1.0, 0.4357080799304952, 1.0, 0.2511122095897182, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.2942553191489362, 1.0, 0.5335917312661498, 1.0, 0.33093525179856115, 0.475963177633822, 0.6270048489369637, 0.5278084714548803, 1.0, 0.2601734489659773, 0.47700127064803044, 1.0, 1.0, 0.23847601128880527, 1.0, 1.0, 0.3165591397849462, 0.2613858165256994, 1.0, 1.0, 1.0, 1.0, 0.36721311475409835, 0.6819787985865724, 1.0, 1.0, 0.35283363802559414, 1.0, 0.6237113402061856, 0.35306704707560627, 1.0, 0.5876436781609196, 0.2384578772013327, 0.238004054967335, 1.0, 0.26312978458251735, 0.47444551591128253, 1.0, 1.0, 0.34037204625439915, 1.0, 0.2542565266742338, 0.3528719397363465, 1.0, 1.0, 0.32308398347865996, 1.0, 0.48610145934676857, 1.0, 1.0, 0.4017595307917889, 0.5252423938482113, 0.29437545653761865, 0.4140593254774482, 1.0, 1.0, 1.0, 0.24578399712952997, 0.35645710746361103, 1.0, 0.2823325635103926, 0.38165369240479696, 1.0, 0.25814467145223635, 0.5286745157614887, 1.0, 1.0, 1.0, 0.4643314822725331, 1.0, 1.0, 0.3257161248793048, 0.48902121324897657, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.38386194029850745, 1.0, 0.4445430346051464, 1.0, 1.0, 1.0, 0.4663865546218488, 1.0, 0.5818399044205496, 1.0, 1.0, 1.0, 0.2446346101602825, 1.0, 0.3189655172413793, 0.29559625876851126, 1.0, 1.0, 1.0, 1.0, 0.3280334728033473, 0.7022739291380223, 1.0, 0.43243243243243246, 0.25040128410914925, 0.3573234779225831, 1.0, 1.0, 0.35801393728222997, 0.5666397415185784, 1.0, 1.0, 0.47652455477603883, 1.0, 1.0, 1.0, 0.33571428571428574, 0.36967886482449586, 1.0, 0.2710592160133445, 0.48303030303030303, 1.0, 1.0, 1.0, 1.0, 1.0, 0.2502067140730941, 0.3243131584383392, 1.0, 0.31137254901960787, 0.306247604446148, 1.0, 1.0, 1.0, 0.2463768115942029, 0.32022932022932027, 1.0, 0.38603988603988604, 1.0, 1.0, 0.3296853625170999, 1.0, 0.35864709636247605, 1.0, 1.0, 1.0, 1.0, 1.0, 0.3152094048493755, 1.0, 1.0, 1.0, 1.0, 0.5807972567509644, 1.0, 0.2477919528949951, 1.0, 0.3471810089020772, 1.0, 0.5194479297365119, 0.3163853028798411, 1.0, 0.3527751646284102, 0.4542994588093806, 1.0, 1.0, 0.31086231538828013, 1.0, 0.29127775989700677, 0.48819444444444443, 0.618066561014263, 0.509004417261298, 0.48590710599444226, 0.48606811145510836, 0.34668989547038326, 0.4341856060606061, 1.0, 1.0, 1.0, 1.0, 1.0, 0.47091412742382277, 0.26705202312138726, 1.0, 1.0, 0.4583756345177665, 1.0, 0.32454526875127737, 0.48540489642184553, 1.0, 1.0, 0.4028197381671702, 0.41747572815533984, 1.0, 1.0, 0.23760330578512398, 1.0, 1.0, 1.0, 0.34391792867611143, 0.5143167814425517, 0.2622950819672131, 0.596128170894526, 0.42869269949066213, 0.2519862089641733, 1.0, 0.5242174629324546, 1.0, 0.2533039647577093, 1.0, 0.2556424581005587, 1.0, 0.3404987102321582, 0.23764119113979754, 0.24411700021992525, 0.2508210180623974, 0.5047858942065492, 1.0, 1.0, 1.0, 0.4086090587857372, 0.38173076923076926, 0.3390064397424103, 1.0, 0.4770544290288154, 1.0, 1.0, 0.5216110019646366, 0.40810048705460134, 1.0, 1.0, 0.5236552171095269, 1.0, 0.4862604540023895, 0.323869610935857, 1.0, 1.0, 1.0, 0.4705119896305897, 1.0, 0.3711526855763428, 0.46581691772885286, 0.5797720797720798, 1.0, 1.0, 1.0, 0.26407309165929854, 0.3624248758818918, 1.0, 1.0, 0.31331708397961444, 0.5058459422283356, 1.0, 0.5043478260869565, 0.4848024316109423, 0.32508109953901315, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.2407079646017699, 0.29851380042462844, 1.0, 0.2457898957497995, 0.5551712210307852, 0.2545454545454546, 0.2576013513513513, 1.0, 1.0, 0.5025445292620866, 1.0, 0.47512437810945274, 1.0, 1.0, 0.30226274907125966, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.2573346593734461, 1.0, 1.0, 0.4695550351288056, 1.0, 1.0, 0.2896174863387978, 0.37201081305884803, 0.3104979384712972, 1.0, 1.0, 0.23846153846153845, 0.31376215407629027, 1.0, 1.0, 1.0, 1.0, 0.2386348307028194, 0.31777231777231774, 1.0, 0.34224841341795104, 1.0, 0.46945701357466063, 0.32570806100217864, 1.0, 0.3194888178913738, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.3150793650793651, 1.0, 0.33558341369334616, 1.0, 1.0, 1.0, 1.0, 0.2925219941348973, 1.0, 1.0, 0.539691943127962, 0.25474642392717817, 0.5089737689829728, 1.0, 1.0, 1.0, 0.5905044510385756, 1.0, 0.30583369612538097, 1.0, 1.0, 1.0, 0.2516258129064532, 0.5467224546722455, 1.0, 0.3634881825590872, 1.0, 1.0, 1.0, 0.53099173553719, 0.45478036175710596, 0.30775477060495326, 0.29541925465838503, 0.45736687357072847, 0.28958333333333336, 0.28749085588880763, 1.0, 1.0, 1.0, 0.40730067243035545, 0.37256135934550033, 1.0, 1.0, 1.0, 0.33670886075949363, 0.3565970453387672, 1.0, 1.0, 0.3022565320665083, 1.0, 1.0, 0.24585365853658536, 1.0, 0.3384010016097299, 0.4939044481054366, 1.0, 0.3190669805264284, 0.3766207038485285, 0.2632089862709749, 1.0, 0.30707456978967496, 1.0, 0.5331274564851207, 1.0, 1.0, 1.0, 0.33476536682088565, 1.0, 1.0, 0.5554844899264471, 0.2908956328645448, 1.0, 0.2451282051282051, 0.48610498456109397, 1.0, 0.5327102803738317, 0.5010060362173038, 0.3992094861660079, 0.3439655172413793, 1.0, 0.30751355538342373, 0.6298076923076923, 0.5359477124183006, 0.24042879019908117, 1.0, 1.0, 1.0, 0.250210970464135, 0.27803379416282636, 1.0, 0.2362952053673703, 1.0, 0.5731176104542626, 1.0, 1.0, 0.36, 1.0, 1.0, 0.2907423839100857, 0.3426074959760865, 1.0, 1.0, 1.0, 0.4349726775956284, 0.6098382749326146, 1.0, 0.4963119072708114, 0.6129518072289156, 0.28415697674418605, 0.3444676409185804, 0.5473647950396142, 1.0, 0.5052064631956912, 0.48482549317147194, 1.0, 0.26059706415965694, 1.0, 1.0, 0.4782241014799154, 1.0, 0.26573889993373095, 1.0, 0.5505328923190004, 0.5173362685745735, 1.0, 1.0, 1.0, 0.27897407762995474, 1.0, 0.3562883931458131, 0.49488175079421104, 1.0, 1.0, 0.24636499524391903, 1.0, 0.5717274357441722, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.29615097159940207, 1.0, 1.0, 1.0, 0.5131500298864314, 0.4785805626598465, 1.0, 0.5237782879632666, 1.0, 0.5003136762860727, 1.0, 0.26487960018173556, 0.49271844660194175, 1.0, 0.51818856718634, 0.3238822246455834, 0.49416990560799556, 1.0, 1.0, 1.0, 0.3230403800475059, 1.0, 0.5245139475908707, 1.0, 1.0, 0.31206860489437355, 0.5050251256281407, 1.0, 1.0, 0.5164512338425382, 0.3149429263230716, 0.45820075209719413, 0.5292397660818714, 1.0, 0.32499999999999996, 0.3586800573888092, 0.45689376816273813, 1.0, 0.23438155136268343, 0.2403232666950234, 1.0, 0.26789587852494573, 1.0, 1.0, 0.2632193494578816, 1.0, 0.3241192411924119, 0.31644706951385737, 1.0, 0.4993394980184941, 0.24279235503725302, 1.0, 1.0, 0.3332552144363721, 0.2471641791044776, 1.0, 1.0, 1.0, 0.46684709066305813, 0.6116207951070336, 0.2385555108068197, 0.3254019292604502, 1.0, 1.0, 0.35238095238095235, 0.337022342869075, 0.367559828185723, 0.31645002241147474, 1.0, 1.0, 1.0, 0.33672546359113525, 1.0, 0.34671280276816613, 1.0, 1.0, 0.36003770028275217, 1.0, 0.28019596758997545, 1.0, 0.33177351427234447, 0.3835511145272867, 0.2928780487804878, 1.0, 1.0, 1.0, 0.510828025477707, 1.0, 0.2426614481409002], "validation": []}, {"column": "Service Available", "alias": "Service Available", "generated": 1, "type": "String", "max": "NoAppend", "min": "DSLAppend", "mean": "", "missing": 0.0, "stddev": "", "outliers": [], "validation": []}]}, "predictionPowerScore": [{"Churn": 1.0, "Contract": 0.0, "Dependents": 0.0, "DeviceProtection": 0.0, "InternetService": 0.0, "MonthlyCharges": 0.1045036141, "MultipleLines": 0.0, "New Charges": 0.1819983199, "OnlineBackup": 0.0, "OnlineSecurity": 0.0, "PaperlessBilling": 0.0, "Partner": 0.0, "PaymentMethod": 0.0, "PhoneService": 0.0, "SeniorCitizen": 0.0, "Service Available": 0.0, "StreamingMovies": 0.0, "StreamingTV": 0.0, "TechSupport": 0.0, "TotalCharges": 0.078636906, "customerID": 0.0, "gender": 0.0, "tenure": 0.2316790568}, {"Churn": 0.0, "Contract": 1.0, "Dependents": 0.0, "DeviceProtection": 0.1324787755, "InternetService": 0.1324787755, "MonthlyCharges": 0.1336610312, "MultipleLines": 0.0, "New Charges": 0.3285499682, "OnlineBackup": 0.1324787755, "OnlineSecurity": 0.1537112968, "PaperlessBilling": 0.0, "Partner": 0.0, "PaymentMethod": 0.1231568454, "PhoneService": 0.0, "SeniorCitizen": 0.0, "Service Available": 0.1324787755, "StreamingMovies": 0.1324787755, "StreamingTV": 0.1324787755, "TechSupport": 0.2078582575, "TotalCharges": 0.1981644541, "customerID": 0.0, "gender": 0.0, "tenure": 0.4088157423}, {"Churn": 0.0, "Contract": 0.0, "Dependents": 1.0, "DeviceProtection": 0.0, "InternetService": 0.0, "MonthlyCharges": 0.0529379812, "MultipleLines": 0.0, "New Charges": 0.0, "OnlineBackup": 0.0, "OnlineSecurity": 0.0, "PaperlessBilling": 0.0, "Partner": 0.3421008511, "PaymentMethod": 0.0, "PhoneService": 0.0, "SeniorCitizen": 0.0, "Service Available": 0.0, "StreamingMovies": 0.0, "StreamingTV": 0.0, "TechSupport": 0.0, "TotalCharges": 0.0453188832, "customerID": 0.0, "gender": 0.0, "tenure": 0.0}, {"Churn": 0.0508945535, "Contract": 0.2002736777, "Dependents": 0.0, "DeviceProtection": 1.0, "InternetService": 0.2603468184, "MonthlyCharges": 0.4971032173, "MultipleLines": 0.0643494851, "New Charges": 0.0769147022, "OnlineBackup": 0.5038689797, "OnlineSecurity": 0.5023303075, "PaperlessBilling": 0.0025865337, "Partner": 0.0779447521, "PaymentMethod": 0.2240796305, "PhoneService": 0.0, "SeniorCitizen": 0.0, "Service Available": 0.2603468184, "StreamingMovies": 0.5735403291, "StreamingTV": 0.5691087246, "TechSupport": 0.5442739101, "TotalCharges": 0.177483329, "customerID": 0.0, "gender": 0.0, "tenure": 0.1773312613}, {"Churn": 0.0756726175, "Contract": 0.1128009697, "Dependents": 0.0545757668, "DeviceProtection": 0.2863912193, "InternetService": 1.0, "MonthlyCharges": 0.7769658444, "MultipleLines": 0.1768821428, "New Charges": 0.0, "OnlineBackup": 0.2863912193, "OnlineSecurity": 0.5273376578, "PaperlessBilling": 0.0823930252, "Partner": 0.0, "PaymentMethod": 0.1015250835, "PhoneService": 0.142028373, "SeniorCitizen": 0.0, "Service Available": 1.0, "StreamingMovies": 0.4899613408, "StreamingTV": 0.4996716973, "TechSupport": 0.5296214257, "TotalCharges": 0.1950438593, "customerID": 0.0, "gender": 0.0, "tenure": 0.0}, {"Churn": 0.0185730084, "Contract": 0.0, "Dependents": 0.0, "DeviceProtection": 0.4640620694, "InternetService": 0.6250831489, "MonthlyCharges": 1.0, "MultipleLines": 0.1718835846, "New Charges": 0.0, "OnlineBackup": 0.4500768721, "OnlineSecurity": 0.4341602111, "PaperlessBilling": 0.0750736959, "Partner": 0.0, "PaymentMethod": 0.1006047859, "PhoneService": 0.0523489805, "SeniorCitizen": 0.0197274673, "Service Available": 0.6250831489, "StreamingMovies": 0.5156816942, "StreamingTV": 0.5200358908, "TechSupport": 0.436601225, "TotalCharges": 0.1432123517, "customerID": 0.0, "gender": 0.0, "tenure": 0.0168679774}, {"Churn": 0.0, "Contract": 0.0797676586, "Dependents": 0.0, "DeviceProtection": 0.194049849, "InternetService": 0.2501146933, "MonthlyCharges": 0.4240081242, "MultipleLines": 1.0, "New Charges": 0.1044462389, "OnlineBackup": 0.1929921954, "OnlineSecurity": 0.130171174, "PaperlessBilling": 0.1329666272, "Partner": 0.1363512842, "PaymentMethod": 0.1020275123, "PhoneService": 0.0277673318, "SeniorCitizen": 0.043928772, "Service Available": 0.2501146933, "StreamingMovies": 0.2347502255, "StreamingTV": 0.2316058164, "TechSupport": 0.130171174, "TotalCharges": 0.2101512738, "customerID": 0.0, "gender": 0.0, "tenure": 0.2363117907}, {"Churn": 0.0, "Contract": 0.0, "Dependents": 0.0, "DeviceProtection": 0.0, "InternetService": 0.0, "MonthlyCharges": 0.0, "MultipleLines": 0.0, "New Charges": 1.0, "OnlineBackup": 0.0, "OnlineSecurity": 0.0, "PaperlessBilling": 0.0, "Partner": 0.0, "PaymentMethod": 0.0, "PhoneService": 0.0, "SeniorCitizen": 0.0, "Service Available": 0.0, "StreamingMovies": 0.0, "StreamingTV": 0.0, "TechSupport": 0.0, "TotalCharges": 0.6547641062, "customerID": 0.0, "gender": 0.0, "tenure": 0.9694053568}, {"Churn": 0.0, "Contract": 0.1500235169, "Dependents": 0.0, "DeviceProtection": 0.5048915826, "InternetService": 0.2727294137, "MonthlyCharges": 0.4747304079, "MultipleLines": 0.0691993944, "New Charges": 0.1108323192, "OnlineBackup": 1.0, "OnlineSecurity": 0.5053568167, "PaperlessBilling": 0.0052504462, "Partner": 0.0595143067, "PaymentMethod": 0.2209544214, "PhoneService": 0.0, "SeniorCitizen": 0.0, "Service Available": 0.2727294137, "StreamingMovies": 0.4171948562, "StreamingTV": 0.4754281175, "TechSupport": 0.5083287957, "TotalCharges": 0.2182563092, "customerID": 0.0, "gender": 0.0, "tenure": 0.1565974051}, {"Churn": 0.0, "Contract": 0.1592326897, "Dependents": 0.0, "DeviceProtection": 0.3550246723, "InternetService": 0.3550246723, "MonthlyCharges": 0.4839400675, "MultipleLines": 0.0, "New Charges": 0.0824738551, "OnlineBackup": 0.3550246723, "OnlineSecurity": 1.0, "PaperlessBilling": 0.0877796668, "Partner": 0.0, "PaymentMethod": 0.1514982186, "PhoneService": 0.0, "SeniorCitizen": 0.0, "Service Available": 0.3550246723, "StreamingMovies": 0.3550246723, "StreamingTV": 0.3550246723, "TechSupport": 0.5664366009, "TotalCharges": 0.1852874633, "customerID": 0.0, "gender": 0.0, "tenure": 0.1154553585}, {"Churn": 0.0, "Contract": 0.1338349116, "Dependents": 0.0, "DeviceProtection": 0.2965414953, "InternetService": 0.2965414953, "MonthlyCharges": 0.1720162693, "MultipleLines": 0.0, "New Charges": 0.0113618516, "OnlineBackup": 0.2965414953, "OnlineSecurity": 0.2965414953, "PaperlessBilling": 1.0, "Partner": 0.0, "PaymentMethod": 0.1831726456, "PhoneService": 0.0, "SeniorCitizen": 0.0, "Service Available": 0.2965414953, "StreamingMovies": 0.2965414953, "StreamingTV": 0.2965414953, "TechSupport": 0.2965414953, "TotalCharges": 0.0948356859, "customerID": 0.0, "gender": 0.0, "tenure": 0.0}, {"Churn": 0.0417174601, "Contract": 0.2603146326, "Dependents": 0.3964116473, "DeviceProtection": 0.1104955386, "InternetService": 0.0, "MonthlyCharges": 0.0307940496, "MultipleLines": 0.1250671197, "New Charges": 0.1374824099, "OnlineBackup": 0.0792829769, "OnlineSecurity": 0.0737233118, "PaperlessBilling": 0.0, "Partner": 1.0, "PaymentMethod": 0.1514947037, "PhoneService": 0.0, "SeniorCitizen": 0.0, "Service Available": 0.0, "StreamingMovies": 0.0980271056, "StreamingTV": 0.0965148404, "TechSupport": 0.0618985563, "TotalCharges": 0.126109543, "customerID": 0.0, "gender": 0.0, "tenure": 0.2753671602}, {"Churn": 0.0, "Contract": 0.0597291602, "Dependents": 0.0, "DeviceProtection": 0.0526124827, "InternetService": 0.0736883015, "MonthlyCharges": 0.0964886158, "MultipleLines": 0.0, "New Charges": 0.0438588867, "OnlineBackup": 0.0526124827, "OnlineSecurity": 0.1665444392, "PaperlessBilling": 0.0169743113, "Partner": 0.0, "PaymentMethod": 1.0, "PhoneService": 0.0, "SeniorCitizen": 0.0, "Service Available": 0.0736883015, "StreamingMovies": 0.0526124827, "StreamingTV": 0.0526124827, "TechSupport": 0.1624651915, "TotalCharges": 0.0384132871, "customerID": 0.0, "gender": 0.0, "tenure": 0.081978425}, {"Churn": 0.0, "Contract": 0.0, "Dependents": 0.0, "DeviceProtection": 0.0, "InternetService": 0.0, "MonthlyCharges": 0.3831399318, "MultipleLines": 1.0, "New Charges": 0.0, "OnlineBackup": 0.0, "OnlineSecurity": 0.0, "PaperlessBilling": 0.0, "Partner": 0.0, "PaymentMethod": 0.0, "PhoneService": 1.0, "SeniorCitizen": 0.0, "Service Available": 0.0, "StreamingMovies": 0.0, "StreamingTV": 0.0, "TechSupport": 0.0, "TotalCharges": 0.0, "customerID": 0.0, "gender": 0.0, "tenure": 0.0}, {"Churn": 0.0, "Contract": 0.0, "Dependents": 0.0, "DeviceProtection": 0.0, "InternetService": 0.0, "MonthlyCharges": 0.0, "MultipleLines": 0.0, "New Charges": 0.0, "OnlineBackup": 0.0, "OnlineSecurity": 0.0, "PaperlessBilling": 0.0, "Partner": 0.0, "PaymentMethod": 0.0, "PhoneService": 0.0, "SeniorCitizen": 1.0, "Service Available": 0.0, "StreamingMovies": 0.0, "StreamingTV": 0.0, "TechSupport": 0.0, "TotalCharges": 0.0, "customerID": 0.0, "gender": 0.0, "tenure": 0.0}, {"Churn": 0.0756726175, "Contract": 0.1128009697, "Dependents": 0.0545757668, "DeviceProtection": 0.2863912193, "InternetService": 1.0, "MonthlyCharges": 0.7769658444, "MultipleLines": 0.1768821428, "New Charges": 0.0, "OnlineBackup": 0.2863912193, "OnlineSecurity": 0.5273376578, "PaperlessBilling": 0.0823930252, "Partner": 0.0, "PaymentMethod": 0.1015250835, "PhoneService": 0.142028373, "SeniorCitizen": 0.0, "Service Available": 1.0, "StreamingMovies": 0.4899613408, "StreamingTV": 0.4996716973, "TechSupport": 0.5296214257, "TotalCharges": 0.1950438593, "customerID": 0.0, "gender": 0.0, "tenure": 0.0}, {"Churn": 0.0, "Contract": 0.1245726967, "Dependents": 0.0, "DeviceProtection": 0.5719356539, "InternetService": 0.4848958001, "MonthlyCharges": 0.6083178079, "MultipleLines": 0.0973177254, "New Charges": 0.0710551067, "OnlineBackup": 0.4681460108, "OnlineSecurity": 0.4190439604, "PaperlessBilling": 0.0, "Partner": 0.0635067994, "PaymentMethod": 0.0, "PhoneService": 0.0, "SeniorCitizen": 0.0, "Service Available": 0.4848958001, "StreamingMovies": 1.0, "StreamingTV": 0.6688544494, "TechSupport": 0.4869711533, "TotalCharges": 0.2059264644, "customerID": 0.0, "gender": 0.0, "tenure": 0.1120807469}, {"Churn": 0.0, "Contract": 0.1241985229, "Dependents": 0.0, "DeviceProtection": 0.5685460301, "InternetService": 0.4959616677, "MonthlyCharges": 0.610206676, "MultipleLines": 0.1005058419, "New Charges": 0.0769690209, "OnlineBackup": 0.4736687351, "OnlineSecurity": 0.4108718217, "PaperlessBilling": 0.0, "Partner": 0.0644514403, "PaymentMethod": 0.0, "PhoneService": 0.0, "SeniorCitizen": 0.0, "Service Available": 0.4959616677, "StreamingMovies": 0.6696561612, "StreamingTV": 1.0, "TechSupport": 0.4874527217, "TotalCharges": 0.2296543376, "customerID": 0.0, "gender": 0.0, "tenure": 0.1300004092}, {"Churn": 0.0, "Contract": 0.190402535, "Dependents": 0.0, "DeviceProtection": 0.5477869782, "InternetService": 0.3916785201, "MonthlyCharges": 0.479980282, "MultipleLines": 0.0, "New Charges": 0.0757929986, "OnlineBackup": 0.3540604098, "OnlineSecurity": 0.5723285344, "PaperlessBilling": 0.0843586083, "Partner": 0.0, "PaymentMethod": 0.1549937117, "PhoneService": 0.0, "SeniorCitizen": 0.0, "Service Available": 0.3916785201, "StreamingMovies": 0.3540604098, "StreamingTV": 0.3540604098, "TechSupport": 1.0, "TotalCharges": 0.1806223076, "customerID": 0.0, "gender": 0.0, "tenure": 0.1364004646}, {"Churn": 0.0, "Contract": 0.0742154176, "Dependents": 0.0, "DeviceProtection": 0.1731762429, "InternetService": 0.0651473002, "MonthlyCharges": 0.2266090286, "MultipleLines": 0.0940944064, "New Charges": 0.3729515677, "OnlineBackup": 0.1602606706, "OnlineSecurity": 0.1129094049, "PaperlessBilling": 0.0, "Partner": 0.0032774794, "PaymentMethod": 0.021745533, "PhoneService": 0.0, "SeniorCitizen": 0.0, "Service Available": 0.0651473002, "StreamingMovies": 0.1491788656, "StreamingTV": 0.1531873297, "TechSupport": 0.1176828933, "TotalCharges": 1.0, "customerID": 0.0, "gender": 0.0, "tenure": 0.5144616861}, {"Churn": 0.0, "Contract": 0.0, "Dependents": 0.0, "DeviceProtection": 0.0, "InternetService": 0.0, "MonthlyCharges": 0.0, "MultipleLines": 0.0, "New Charges": 0.0, "OnlineBackup": 0.0, "OnlineSecurity": 0.0, "PaperlessBilling": 0.0, "Partner": 0.0, "PaymentMethod": 0.0, "PhoneService": 0.0, "SeniorCitizen": 0.0, "Service Available": 0.0, "StreamingMovies": 0.0, "StreamingTV": 0.0, "TechSupport": 0.0, "TotalCharges": 0.0, "customerID": 1.0, "gender": 0.0, "tenure": 0.0}, {"Churn": 0.0, "Contract": 0.0, "Dependents": 0.0, "DeviceProtection": 0.0, "InternetService": 0.0, "MonthlyCharges": 0.0, "MultipleLines": 0.0, "New Charges": 0.0, "OnlineBackup": 0.0, "OnlineSecurity": 0.0, "PaperlessBilling": 0.0, "Partner": 0.0, "PaymentMethod": 0.0, "PhoneService": 0.0, "SeniorCitizen": 0.0, "Service Available": 0.0, "StreamingMovies": 0.0, "StreamingTV": 0.0027513077, "TechSupport": 0.0, "TotalCharges": 0.000541107, "customerID": 0.0, "gender": 1.0, "tenure": 0.0}, {"Churn": 0.0799659085, "Contract": 0.3236292241, "Dependents": 0.0081087683, "DeviceProtection": 0.0978905729, "InternetService": 0.0, "MonthlyCharges": 0.0197705524, "MultipleLines": 0.0791590375, "New Charges": 0.9542386286, "OnlineBackup": 0.0875949622, "OnlineSecurity": 0.0742741383, "PaperlessBilling": 0.0, "Partner": 0.0838970473, "PaymentMethod": 0.1089953339, "PhoneService": 0.0, "SeniorCitizen": 0.0, "Service Available": 0.0, "StreamingMovies": 0.0586024461, "StreamingTV": 0.0561383592, "TechSupport": 0.0693137664, "TotalCharges": 0.5211348992, "customerID": 0.0, "gender": 0.0, "tenure": 1.0}], "HasBasicStats": 1, "functionsApplied": [{"functionName": "formula", "applyOn": [{"columnName": "MonthlyCharges", "type": "real", "min": "18.3", "max": "118.8", "mean": "64.8"}, {"columnName": "TotalCharges", "type": "real", "min": "18.8", "max": "8684.8", "mean": "2283.3"}], "functionsData": [{"operand1": "MonthlyCharges", "operator": "div", "operand2": "TotalCharges", "outputColumn": "newColumn", "useNext": 0, "column": [{"Columndata": {"columnName": "MonthlyCharges", "type": "real", "min": "18.3", "max": "118.8", "mean": "64.8"}}, {"Columndata": {"columnName": "TotalCharges", "type": "real", "min": "18.8", "max": "8684.8", "mean": "2283.3"}}, {"Columndata": ""}], "newColumnName": "New Charges"}]}, {"functionName": "appendvalue", "applyOn": [{"columnName": "InternetService", "type": "String", "min": "DSL", "max": "No", "mean": "-"}], "functionsData": [{"columnName": "InternetService", "appendValue": "Append", "newColumnName": "Service Available", "asNewColumn": 1, "Column": {"columnName": "InternetService", "type": "String", "min": "DSL", "max": "No", "mean": "-"}}]}], "functionChanges": [{"columnName": "MonthlyCharges", "functionName": "Formula", "Type": "real", "Parameters": [{"operand1": "MonthlyCharges", "operator": "div", "operand2": "TotalCharges", "outputColumn": "newColumn", "useNext": 0, "column": [{"Columndata": {"columnName": "MonthlyCharges", "type": "real", "min": "18.3", "max": "118.8", "mean": "64.8"}}, {"Columndata": {"columnName": "TotalCharges", "type": "real", "min": "18.8", "max": "8684.8", "mean": "2283.3"}}, {"Columndata": ""}], "newColumnName": "New Charges"}]}, {"columnName": "TotalCharges", "functionName": "Formula", "Type": "real", "Parameters": [{"operand1": "MonthlyCharges", "operator": "div", "operand2": "TotalCharges", "outputColumn": "newColumn", "useNext": 0, "column": [{"Columndata": {"columnName": "MonthlyCharges", "type": "real", "min": "18.3", "max": "118.8", "mean": "64.8"}}, {"Columndata": {"columnName": "TotalCharges", "type": "real", "min": "18.8", "max": "8684.8", "mean": "2283.3"}}, {"Columndata": ""}], "newColumnName": "New Charges"}]}, {"columnName": "InternetService", "functionName": "Append Value", "Type": "String", "Parameters": [{"columnName": "InternetService", "appendValue": "Append", "newColumnName": "Service Available", "asNewColumn": 1, "Column": {"columnName": "InternetService", "type": "String", "min": "DSL", "max": "No", "mean": "-"}}]}], "fileheader": [{"field": "customerID", "alias": "customerID", "generated": 0, "position": 1, "type": "String"}, {"field": "gender", "alias": "gender", "generated": 0, "position": 2, "type": "String"}, {"field": "SeniorCitizen", "alias": "SeniorCitizen", "generated": 0, "position": 3, "type": "numeric"}, {"field": "Partner", "alias": "Partner", "generated": 0, "position": 4, "type": "String"}, {"field": "Dependents", "alias": "Dependents", "generated": 0, "position": 5, "type": "String"}, {"field": "tenure", "alias": "tenure", "generated": 0, "position": 6, "type": "numeric"}, {"field": "PhoneService", "alias": "PhoneService", "generated": 0, "position": 7, "type": "String"}, {"field": "MultipleLines", "alias": "MultipleLines", "generated": 0, "position": 8, "type": "String"}, {"field": "InternetService", "alias": "InternetService", "generated": 0, "position": 9, "type": "String"}, {"field": "OnlineSecurity", "alias": "OnlineSecurity", "generated": 0, "position": 10, "type": "String"}, {"field": "OnlineBackup", "alias": "OnlineBackup", "generated": 0, "position": 11, "type": "String"}, {"field": "DeviceProtection", "alias": "DeviceProtection", "generated": 0, "position": 12, "type": "String"}, {"field": "TechSupport", "alias": "TechSupport", "generated": 0, "position": 13, "type": "String"}, {"field": "StreamingTV", "alias": "StreamingTV", "generated": 0, "position": 14, "type": "String"}, {"field": "StreamingMovies", "alias": "StreamingMovies", "generated": 0, "position": 15, "type": "String"}, {"field": "Contract", "alias": "Contract", "generated": 0, "position": 16, "type": "String"}, {"field": "PaperlessBilling", "alias": "PaperlessBilling", "generated": 0, "position": 17, "type": "String"}, {"field": "PaymentMethod", "alias": "PaymentMethod", "generated": 0, "position": 18, "type": "String"}, {"field": "MonthlyCharges", "alias": "MonthlyCharges", "generated": 0, "position": 19, "type": "real"}, {"field": "TotalCharges", "alias": "TotalCharges", "generated": 0, "position": 20, "type": "real"}, {"field": "Churn", "alias": "Churn", "generated": 0, "position": 21, "type": "String"}, {"field": "New Charges", "alias": "New Charges", "generated": 1, "position": 22, "type": "real"}, {"field": "Service Available", "alias": "Service Available", "generated": 1, "position": 23, "type": "String"}]}))
	#operationPostExecutionHook(datapreparation)

except Exception as ex: 
	logging.error(ex)


***WRITING DATAFRAME***

In [ ]:
#%run dp1Hooks.ipynb
try:
	#sinkPreExecutionHook()

	retailchurn = NumtraConnector.fetch(spark, "{'samplefile': '/FileStore/platform/sampleData/64a66aea1ba55170f7a206bd/part-00000-deca3486-5ecf-443b-b252-b1452830f502-c000.csv', 'samplecount': 715, 'originalcount': 7043, 'orignalKey': '/FileStore/platform/uploadedSourceFiles/CustomerChurn.csv', 'pathOnly': '/Retail Churn', 'project_id': '649c0ff6e61a96675bcdf706', 'parent_id': '649c0ff6e61a96675bcdf706', 'original_schema': [{'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'customerID', 'alias': 'customerID', 'type': 'String', 'position': '0'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'gender', 'alias': 'gender', 'type': 'String', 'position': '1'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'SeniorCitizen', 'alias': 'SeniorCitizen', 'type': 'numeric', 'position': '2'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Partner', 'alias': 'Partner', 'type': 'String', 'position': '3'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Dependents', 'alias': 'Dependents', 'type': 'String', 'position': '4'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'tenure', 'alias': 'tenure', 'type': 'numeric', 'position': '5'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'PhoneService', 'alias': 'PhoneService', 'type': 'String', 'position': '6'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'MultipleLines', 'alias': 'MultipleLines', 'type': 'String', 'position': '7'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'InternetService', 'alias': 'InternetService', 'type': 'String', 'position': '8'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'OnlineSecurity', 'alias': 'OnlineSecurity', 'type': 'String', 'position': '9'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'OnlineBackup', 'alias': 'OnlineBackup', 'type': 'String', 'position': '10'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'DeviceProtection', 'alias': 'DeviceProtection', 'type': 'String', 'position': '11'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'TechSupport', 'alias': 'TechSupport', 'type': 'String', 'position': '12'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'StreamingTV', 'alias': 'StreamingTV', 'type': 'String', 'position': '13'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'StreamingMovies', 'alias': 'StreamingMovies', 'type': 'String', 'position': '14'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Contract', 'alias': 'Contract', 'type': 'String', 'position': '15'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'PaperlessBilling', 'alias': 'PaperlessBilling', 'type': 'String', 'position': '16'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'PaymentMethod', 'alias': 'PaymentMethod', 'type': 'String', 'position': '17'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'MonthlyCharges', 'alias': 'MonthlyCharges', 'type': 'real', 'position': '18'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'TotalCharges', 'alias': 'TotalCharges', 'type': 'real', 'position': '19'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Churn', 'alias': 'Churn', 'type': 'String', 'position': '20'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'New Charges', 'alias': 'New Charges', 'type': 'real', 'position': '22'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Service Available', 'alias': 'Service Available', 'type': 'String', 'position': '23'}], 'actual_schema': [{'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'customerID', 'alias': 'customerID', 'type': 'String', 'position': '0'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'gender', 'alias': 'gender', 'type': 'String', 'position': '1'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'SeniorCitizen', 'alias': 'SeniorCitizen', 'type': 'numeric', 'position': '2'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Partner', 'alias': 'Partner', 'type': 'String', 'position': '3'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Dependents', 'alias': 'Dependents', 'type': 'String', 'position': '4'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'tenure', 'alias': 'tenure', 'type': 'numeric', 'position': '5'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'PhoneService', 'alias': 'PhoneService', 'type': 'String', 'position': '6'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'MultipleLines', 'alias': 'MultipleLines', 'type': 'String', 'position': '7'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'InternetService', 'alias': 'InternetService', 'type': 'String', 'position': '8'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'OnlineSecurity', 'alias': 'OnlineSecurity', 'type': 'String', 'position': '9'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'OnlineBackup', 'alias': 'OnlineBackup', 'type': 'String', 'position': '10'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'DeviceProtection', 'alias': 'DeviceProtection', 'type': 'String', 'position': '11'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'TechSupport', 'alias': 'TechSupport', 'type': 'String', 'position': '12'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'StreamingTV', 'alias': 'StreamingTV', 'type': 'String', 'position': '13'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'StreamingMovies', 'alias': 'StreamingMovies', 'type': 'String', 'position': '14'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Contract', 'alias': 'Contract', 'type': 'String', 'position': '15'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'PaperlessBilling', 'alias': 'PaperlessBilling', 'type': 'String', 'position': '16'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'PaymentMethod', 'alias': 'PaymentMethod', 'type': 'String', 'position': '17'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'MonthlyCharges', 'alias': 'MonthlyCharges', 'type': 'real', 'position': '18'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'TotalCharges', 'alias': 'TotalCharges', 'type': 'real', 'position': '19'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Churn', 'alias': 'Churn', 'type': 'String', 'position': '20'}], 'server': 'https://ml.numtra.com:443', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'delimiter': ',', 'file_type': 'Delimeted', 'filename': 'DPData.csv', 'token': '', 'domain': '', 'is_header': 'Use Header Line', 'url': '/FileStore/platform/uploadedSourceFiles/part-00000-7b1efa7b-18ef-43bf-8eaa-299938f1582c-c000.csv', 'results_url': 'http://ml.numtra.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
